In [75]:
import pandas as pd
import numpy as np
import datetime as dt
import joblib

In [76]:
# library for models
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
#import matplotlib as mpl
#import matplotlib.pyplot as plt
from sklearn.calibration import CalibratedClassifierCV
from imblearn.over_sampling import SMOTE

In [77]:
nhts_hh = pd.read_csv('../Model_inputs/NHTS/hhpub.csv', header=0, sep=',')

In [78]:
nhts_hh_CA= nhts_hh.query('HH_CBSA ==("31080","40140","40900","41740","41860","41940")')

In [79]:
#nhts_hh_sf= nhts_hh.query('HH_CBSA ==("41860","41940")')

In [80]:
nhts_hh_CA= nhts_hh_CA[['HBHUR','HH_HISP', 'HOMEOWN',
'HBPPOPDN',
'HHFAMINC',
'HHSIZE',
'HHVEHCNT',
'HH_RACE',
'WEBUSE17',
'LIF_CYC',
'WRKCOUNT',
'HH_CBSA']]

In [81]:
nhts_hh_CA=nhts_hh_CA.query('HBHUR !="-9" \
                            and HBPPOPDN !="-9" \
                            and HHFAMINC != (-7,-8, -9) \
                            and HH_RACE != (-7,-8)\
                            and WEBUSE17 != (-7,-9)\
                            and HH_HISP != (-7,-8)\
                            and HOMEOWN != (-7,-8)\
                            and LIF_CYC != (-9)')

In [82]:
def income_est(HHFAMINC):
    if HHFAMINC == 1:
        return np.random.randint(0,10000)
    elif HHFAMINC == 2:
        return np.random.randint(10000,14999)
    elif HHFAMINC == 3:
        return np.random.randint(15000,24999)
    elif HHFAMINC == 4:
        return np.random.randint(25000,34999)
    elif HHFAMINC == 5:
        return np.random.randint(35000,49999)
    elif HHFAMINC == 6:
        return np.random.randint(50000,74999)
    elif HHFAMINC == 7:
        return np.random.randint(75000,99999)
    elif HHFAMINC == 8:
        return np.random.randint(100000,124999)
    elif HHFAMINC == 9:
        return np.random.randint(125000,149999)
    elif HHFAMINC == 10:
        return np.random.randint(150000,199999)
    elif HHFAMINC == 11:
        return np.random.randint(200000,1000000)
nhts_hh_CA['income_est']=nhts_hh_CA['HHFAMINC'].apply(income_est)    

In [83]:
def web_class(WEBUSE17):
    if WEBUSE17 in [1]: 
        return 1 # frequenct use (more than a few time a week)
    elif WEBUSE17 in [2,3,4]:
        return 2 # often use (more than a few times a year)
    else:
        return 0 # (never)
nhts_hh_CA['WEBUSE17']=nhts_hh_CA['WEBUSE17'].apply(web_class)    

In [84]:
def race_class(HH_RACE):
    if HH_RACE == 1: 
        return 1 # white
    elif HH_RACE == 2:
        return 2 # black
    elif HH_RACE == 3:
        return 3 # Asian
    else:
        return 4 # others
nhts_hh_CA['HH_RACE']=nhts_hh_CA['HH_RACE'].apply(race_class)    

In [85]:
def urban_class(HBHUR):
    if HBHUR == "R": 
        return 1 # rural 
    else:
        return 0 # others
nhts_hh_CA['HBHUR']=nhts_hh_CA['HBHUR'].apply(urban_class)   

In [86]:
def home_class(HOMEOWN):
    if HOMEOWN == 1: 
        return 1
    else:
        return 0
nhts_hh_CA['HOMEOWN']=nhts_hh_CA['HOMEOWN'].apply(home_class)   

In [87]:
def hisp_class(HH_HISP):
    if HH_HISP == 1: 
        return 1
    else:
        return 0
nhts_hh_CA['HH_HISP']=nhts_hh_CA['HH_HISP'].apply(hisp_class)   

In [88]:
def child_class(LIF_CYC):
    if LIF_CYC in [1,2,9,10]: 
        return 0 # adult only
    else:
        return 1 # child 
nhts_hh_CA['Child']=nhts_hh_CA['LIF_CYC'].apply(child_class)   

In [89]:
# class variables 
cat_vars=['HH_RACE']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(nhts_hh_CA[var], prefix=var)
    nhts_hh_CA_temp=nhts_hh_CA.join(cat_list)
    nhts_hh_CA=nhts_hh_CA_temp
cat_vars=['HH_RACE']
data_vars=nhts_hh_CA.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
nhts_hh_CA_final=nhts_hh_CA[to_keep]
nhts_hh_CA_final.columns.values

array(['HBHUR', 'HH_HISP', 'HOMEOWN', 'HBPPOPDN', 'HHFAMINC', 'HHSIZE',
       'HHVEHCNT', 'WEBUSE17', 'LIF_CYC', 'WRKCOUNT', 'HH_CBSA',
       'income_est', 'Child', 'HH_RACE_1', 'HH_RACE_2', 'HH_RACE_3',
       'HH_RACE_4'], dtype=object)

In [90]:
# income scale 
nhts_hh_CA_final['income_est']=nhts_hh_CA_final['income_est']/1000

/var/folders/kr/2j58542j2_7chf9_h8bmr30mb0_4n0/T/ipykernel_88076/1608345060.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts_hh_CA_final['income_est']=nhts_hh_CA_final['income_est']/1000


In [111]:
# create training & test set : Train data: Non-SF data + 30% of SF data
#                              Test data: 70% of SF data
nhts_hh_nonSF=nhts_hh_CA_final.query('HH_CBSA ==("31080","40140","40900","41740")')
nhts_hh_SF=nhts_hh_CA_final.query('HH_CBSA ==("41860","41940")')
X_nonsf=nhts_hh_nonSF[['HOMEOWN', 'WRKCOUNT', 'income_est','HHVEHCNT', 'HH_RACE_1']]
Y_nonsf=nhts_hh_nonSF['WEBUSE17']
X_sf=nhts_hh_SF[['HOMEOWN', 'WRKCOUNT', 'income_est','HHVEHCNT', 'HH_RACE_1']]
Y_sf=nhts_hh_SF['WEBUSE17']
trainX, testX, trainY, testY = train_test_split(X_sf, Y_sf, test_size = 0.7)

In [112]:
trainY.value_counts()

1    875
2     38
0     23
Name: WEBUSE17, dtype: int64

In [91]:
#nhts_hh_CA_final.head()

In [110]:
trainX=pd.concat([trainX,X_nonsf])
trainY=pd.concat([trainY,Y_nonsf])

In [95]:
# SMOTE approach for lower sample issue for less frequent data
oversample = SMOTE()
trainX,trainY = oversample.fit_resample(trainX,trainY)

In [108]:
trainY.value_counts()

1    10488
2    10488
0    10488
Name: WEBUSE17, dtype: int64

In [97]:
# initial model
model_int = LogisticRegression(solver='newton-cg', multi_class='multinomial')
model_int.fit(trainX, trainY)
y_pred = model_int.predict(testX)

In [98]:
print('Accuracy for initial model: {:.2f}'.format(accuracy_score(testY, y_pred)))
print('Error rate for initial model: {:.2f}'.format(1 - accuracy_score(testY, y_pred)))

Accuracy for initial model: 0.72
Error rate for initial model: 0.28


In [99]:
print ('initial confusion matirix)', confusion_matrix(testY.tolist(), y_pred))

initial confusion matirix) [[  36    5   12]
 [ 157 1498  393]
 [  38   15   31]]


In [100]:
# CalibratedclassifierCV approach to improve the model 
testX_SM,testY_SM = oversample.fit_resample(testX,testY)
model_cal = CalibratedClassifierCV(model_int, cv='prefit')
model_cal.fit(testX_SM,testY_SM)

CalibratedClassifierCV(base_estimator=LogisticRegression(multi_class='multinomial',
                                                         solver='newton-cg'),
                       cv='prefit')

In [101]:
y_pred_cal = model_cal.predict(testX)

In [102]:
print('Accuracy for calibrated model: {:.2f}'.format(accuracy_score(testY, y_pred_cal)))
print('Error rate for calibrated model: {:.2f}'.format(1 - accuracy_score(testY, y_pred_cal)))

Accuracy for calibrated model: 0.78
Error rate for calibrated model: 0.22


In [103]:
print ('calibrated confusion matrix', confusion_matrix(testY, y_pred_cal))

calibrated confusion matrix [[  43    5    5]
 [ 242 1659  147]
 [  55   19   10]]


In [104]:
print('coef/intercept for initial M',model_int.coef_, model_int.intercept_)

coef/intercept for initial M [[ 0.23718163 -0.95164719 -0.01104132 -0.30025234 -0.08248763]
 [-0.17227021  1.22006027  0.00737925  0.18280079  0.53940724]
 [-0.06491142 -0.26841308  0.00366208  0.11745155 -0.45691961]] [ 1.14795418 -1.54419785  0.39624367]


In [105]:
filename = '../Simulation/webuse_model.sav'
joblib.dump(model_cal, filename)

['webuse_model.sav']

array([1, 1, 1, ..., 1, 1, 0])

In [106]:
testX.dtypes

HOMEOWN         int64
WRKCOUNT        int64
income_est    float64
HHVEHCNT        int64
HH_RACE_1       uint8
dtype: object